# Preparación de entorno y dataset

In [ ]:
! pip install pyarrow
! pip install -q kaggle
! pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
pd.options.display.float_format = '{:20,.4f}'.format # suprimimos la notacion cientifica en los outputs

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hangyeolkoo","key":"4032f7e1f591049aec886c05d7504de2"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/train.parq

100% 564M/564M [00:24<00:00, 27.2MB/s]
100% 564M/564M [00:24<00:00, 23.8MB/s]


In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/test.parq

 99% 162M/164M [00:07<00:00, 24.0MB/s]
100% 164M/164M [00:07<00:00, 22.3MB/s]


In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/shodan_df_hashed.csv

 95% 2.00M/2.11M [00:00<00:00, 2.96MB/s]
100% 2.11M/2.11M [00:00<00:00, 2.69MB/s]


In [ ]:
! unzip train.parq.zip

Archive:  train.parq.zip
  inflating: train.parq              


In [ ]:
! unzip test.parq.zip

Archive:  test.parq.zip
  inflating: test.parq               


In [ ]:
! unzip shodan_df_hashed.csv.zip

Archive:  shodan_df_hashed.csv.zip
  inflating: shodan_df_hashed.csv    


## Manejo de train y test dataset

In [ ]:
train = pd.read_parquet("train.parq",engine="auto")
test = pd.read_parquet("test.parq",engine="auto")

**Imputo los valores nulos**

Por cuestiones de perfomance se decidió crear un propio Imputer, que rellena los NaNs con los datos más frecuentes. Esto se llevó a cabo sólo en dataset de test, para no utilizar informaciones erróneas para el entrenamiento.

In [ ]:
class MostFrequentImputer:
    def __init__(self):
        self.most_frequent_values = None

    def fit(self, df):
        self.most_frequent_values = {col: df[col].mode().iloc[0] for col in df.columns}
        return self

    def transform(self, df):
        for col, value in self.most_frequent_values.items():
            df[col] = df[col].fillna(value)
        return df

In [ ]:
imputer = MostFrequentImputer()

test_columns = test.columns

imputer.fit(test)
imputer.transform(test)

test.isna().sum().sum()

0

In [ ]:
train.head()

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,"34,011.0000",Host Europe GmbH,TR,"47,721.0000",Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,"34,011.0000",Host Europe GmbH,TR,"47,721.0000",Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:exploit,2,4637,0


In [ ]:
train.isna().sum().sum()

822426

In [ ]:
train['label'].value_counts()

0    60594448
1     1035237
Name: label, dtype: int64

In [ ]:
train = train.dropna()

In [ ]:
train['label'].value_counts()

0    60135084
1     1026672
Name: label, dtype: int64

In [ ]:
train = train.drop(columns=['watcher_as_name','attacker_as_name'])
test = test.drop(columns=['watcher_as_name','attacker_as_name'])
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 9 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
dtypes: category(3), datetime64[ns, UTC](1), float32(2), int32(2), int8(1)
memory usage: 2.2 GB


Las columnas que aportan informaciones equivalentes a otra ya existente fueron descartadas.

## Feature engineering

In [ ]:
train['day_of_week'] = train['attack_time'].dt.day_name()
train['hour'] = train['attack_time'].dt.hour

test['day_of_week'] = test['attack_time'].dt.day_name()
test['hour'] = test['attack_time'].dt.hour

In [ ]:
def map_day_to_sin(day):
  list_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
  for i in range(7):
    if day == list_days[i]:
      return np.sin(2*np.pi*i/7)
  return day

In [ ]:
def map_hour_to_sin(hour):
  return np.sin(2*np.pi*hour/7)

In [ ]:
train['sin_day_of_week'] = train['day_of_week'].map(map_day_to_sin)
train['sin_hour'] = train['hour'].map(map_hour_to_sin)

test['sin_day_of_week'] = test['day_of_week'].map(map_day_to_sin)
test['sin_hour'] = test['hour'].map(map_hour_to_sin)

In [ ]:
train = train.drop(columns=['day_of_week','hour'])

test = test.drop(columns=['day_of_week','hour'])

In [ ]:
train['sin_day_of_week'] = train['sin_day_of_week'].astype('float')

test['sin_day_of_week'] = test['sin_day_of_week'].astype('float')

In [ ]:
train[['service', 'threat_type']] = train['attack_type'].str.split(':', expand=True)

test[['service', 'threat_type']] = test['attack_type'].str.split(':', expand=True)

In [ ]:
train['same_country'] = (train['attacker_country'].astype('str') == train['watcher_country'].astype('str')).astype(int)
test['same_country'] = (test['attacker_country'].astype('str') == test['watcher_country'].astype('str')).astype(int)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 14 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
 9   sin_day_of_week    float64            
 10  sin_hour           float64            
 11  service            object             
 12  threat_type        object             
 13  same_country       int64              
dtypes: category(3), datetime64[ns, UTC](1), float32(2), float64(2), int32(2), int64(1), int8(1), object(2)
memory usage: 4.4+ GB


In [ ]:
train['service'] = train['service'].astype('category')
train['threat_type'] = train['threat_type'].astype('category')
train['same_country'] = train['same_country'].astype('category')

test['service'] = test['service'].astype('category')
test['threat_type'] = test['threat_type'].astype('category')
test['same_country'] = test['same_country'].astype('category')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 14 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
 9   sin_day_of_week    float64            
 10  sin_hour           float64            
 11  service            category           
 12  threat_type        category           
 13  same_country       category           
dtypes: category(6), datetime64[ns, UTC](1), float32(2), float64(2), int32(2), int8(1)
memory usage: 3.2 GB


In [ ]:
train = train.drop(columns=['attack_time'])

test = test.drop(columns=['attack_time'])

In [ ]:
train.head()

,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,attacker_ip_enum,label,sin_day_of_week,sin_hour,service,threat_type,same_country
0,DE,"34,011.0000",TR,"47,721.0000",http:exploit,0,6466,0,0.0000,-0.0000,http,exploit,0
1,DE,"34,011.0000",TR,"47,721.0000",http:spam,0,6466,0,0.0000,-0.0000,http,spam,0
2,DE,"20,886.0000",DE,"51,167.0000",http:bruteforce,2,4637,0,0.0000,-0.0000,http,bruteforce,1
3,DE,"20,886.0000",DE,"51,167.0000",http:spam,2,4637,0,0.0000,-0.0000,http,spam,1
4,DE,"20,886.0000",DE,"51,167.0000",http:exploit,2,4637,0,0.0000,-0.0000,http,exploit,1


In [ ]:
train['attack_count_per_ip'] = train.groupby('attacker_ip_enum')['attacker_ip_enum'].transform('count')

test['attack_count_per_ip'] = test.groupby('attacker_ip_enum')['attacker_ip_enum'].transform('count')

In [ ]:
train_grouped_by_ip_enum = train.groupby('attacker_ip_enum').agg({
    'attack_count_per_ip' : 'first',
    'watcher_country': lambda x: x.mode().iat[0],
    'watcher_as_num': lambda x: x.mode().iat[0],
    'attacker_country': lambda x: x.mode().iat[0],
    'attacker_as_num': lambda x: x.mode().iat[0],
    'attack_type': lambda x: x.mode().iat[0],
    'watcher_uuid_enum': lambda x: x.mode().iat[0],
    'label': lambda x: x.mode().iat[0],
    'sin_day_of_week': 'mean',
    'sin_hour': 'mean',
    'service': lambda x: x.mode().iat[0],
    'threat_type': lambda x: x.mode().iat[0],
    'same_country': lambda x: x.mode().iat[0]
}).reset_index()

test_grouped_by_ip_enum = test.groupby('attacker_ip_enum').agg({
    'attack_count_per_ip' : 'first',
    'watcher_country': lambda x: x.mode().iat[0],
    'watcher_as_num': lambda x: x.mode().iat[0],
    'attacker_country': lambda x: x.mode().iat[0],
    'attacker_as_num': lambda x: x.mode().iat[0],
    'attack_type': lambda x: x.mode().iat[0],
    'watcher_uuid_enum': lambda x: x.mode().iat[0],
    'sin_day_of_week': 'mean',
    'sin_hour': 'mean',
    'service': lambda x: x.mode().iat[0],
    'threat_type': lambda x: x.mode().iat[0],
    'same_country': lambda x: x.mode().iat[0]
}).reset_index()

train_grouped_by_ip_enum.head()

,attacker_ip_enum,attack_count_per_ip,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,label,sin_day_of_week,sin_hour,service,threat_type,same_country
0,0,52,US,"14,061.0000",US,"14,618.0000",http:scan,18401,0,-0.4339,0.0816,http,scan,0
1,1,30,US,"3,303.0000",BR,"27,699.0000",http:exploit,8646,0,0.7818,0.1567,http,exploit,0
2,2,58,DE,"12,897.0000",DE,"3,320.0000",http:scan,24879,0,0.0890,0.0876,http,scan,1
3,3,22,DE,"12,897.0000",VE,"21,826.0000",http:exploit,24879,0,-0.1423,0.2312,http,exploit,0
4,4,68,US,"396,982.0000",SA,"25,019.0000",http:exploit,1828,0,-0.2439,0.2687,http,exploit,0


## Manejo de shodan_info

In [ ]:
shodan_df = pd.read_csv('shodan_df_hashed.csv')

In [ ]:
shodan_df.head()

,shodan_info,attacker_ip_enum
0,{},5915
1,"{'22/tcp': {'headers_hash': None, 'jarm': None...",3325
2,{},8416
3,{},1213
4,{},9185


In [ ]:
len(shodan_df) == len(shodan_df['attacker_ip_enum'].unique())

True

In [ ]:
import ast
from collections import Counter

port_counter = Counter()
ip_port_list = {}

for index, row in shodan_df.iterrows():
  shodan_info = ast.literal_eval(row['shodan_info'])
  ip_port_list[int(row['attacker_ip_enum'])] = list(shodan_info)
  port_counter.update(shodan_info.keys())

In [ ]:
ip_port_count = {}
for ip in ip_port_list:
  ip_port_count[ip] = len(ip_port_list[ip])

In [ ]:
len(port_counter.keys())

2297

In [ ]:
port_counter.most_common()[15]

('25/tcp', 1873)

In [ ]:
top_port = set()
for port in port_counter.most_common(15):
  top_port.add(port[0])

In [ ]:
for ip in ip_port_list:
  for port in ip_port_list[ip]:
    if port not in top_port:
      ip_port_list[ip].remove(port)

In [ ]:
train_grouped_by_ip_enum['port_count'] = train_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_count)
train_grouped_by_ip_enum['port_list'] = train_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_list)

test_grouped_by_ip_enum['port_count'] = test_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_count)
test_grouped_by_ip_enum['port_list'] = test_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_list)

## Dataset resultante

En resumen, se crearon las siguientes nuevas features:
* Cantidad de ataques por attacker_ip_enum
* Seno de día de semana(siendo 0 lunes, 6 domingo)
* Seno de hora de ataque
* Servicio atacado
* Tipo de amenaza (spam, scam, etc.)
* Si el país atacante y atacado coinciden
* Cantidad de puertos

Este último fue creado teniendo en cuenta que en caso de coincidencia, será más probable que el atacante no esté utilizando un VPN, que generalmente cambia la ubicación.

In [ ]:
train_grouped_by_ip_enum.head()

,attacker_ip_enum,attack_count_per_ip,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,label,sin_day_of_week,sin_hour,service,threat_type,same_country,port_count,port_list
0,0,52,US,"14,061.0000",US,"14,618.0000",http:scan,18401,0,-0.4339,0.0816,http,scan,0,0,[]
1,1,30,US,"3,303.0000",BR,"27,699.0000",http:exploit,8646,0,0.7818,0.1567,http,exploit,0,0,[]
2,2,58,DE,"12,897.0000",DE,"3,320.0000",http:scan,24879,0,0.0890,0.0876,http,scan,1,0,[]
3,3,22,DE,"12,897.0000",VE,"21,826.0000",http:exploit,24879,0,-0.1423,0.2312,http,exploit,0,0,[]
4,4,68,US,"396,982.0000",SA,"25,019.0000",http:exploit,1828,0,-0.2439,0.2687,http,exploit,0,0,[]


## Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = train_grouped_by_ip_enum['label']
X = train_grouped_by_ip_enum.drop(columns=['label'])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)

    if (a_set & b_set):
        print(a_set & b_set)
    else:
        print("No common elements")

common_member(X_train['attacker_ip_enum'],X_valid['attacker_ip_enum'])

No common elements


In [ ]:
X_train = X_train.set_index('attacker_ip_enum')
X_valid = X_valid.set_index('attacker_ip_enum')

X_test = test_grouped_by_ip_enum.set_index('attacker_ip_enum')

## Encodeo los datos

In [ ]:
from category_encoders import BinaryEncoder

In [ ]:
categorical_features = ['watcher_country','watcher_as_num','attacker_country','attacker_as_num','attack_type','watcher_uuid_enum','service','threat_type']

In [ ]:
be = BinaryEncoder(cols=categorical_features, return_df=True)

be.fit(train_grouped_by_ip_enum.loc[:,categorical_features])

X_train_categories_encoded = be.transform(X_train.loc[:,categorical_features])
X_valid_categories_encoded = be.transform(X_valid.loc[:,categorical_features])

X_test_categories_encoded = be.transform(X_test.loc[:,categorical_features])

In [ ]:
X_train_encoded = X_train.drop(columns=categorical_features)
X_valid_encoded = X_valid.drop(columns=categorical_features)

X_test_encoded = X_test.drop(columns=categorical_features)

X_train_encoded = pd.concat([X_train_encoded,X_train_categories_encoded],axis=1)
X_valid_encoded = pd.concat([X_valid_encoded,X_valid_categories_encoded],axis=1)

X_test_encoded = pd.concat([X_test_encoded,X_test_categories_encoded],axis=1)

**Hago un One Hot Encoding Manual**

In [ ]:
for port in top_port:
  X_train_encoded[port] = X_train_encoded['port_list'].map(lambda port_list: int(port in port_list))
  X_valid_encoded[port] = X_valid_encoded['port_list'].map(lambda port_list: int(port in port_list))
  X_test_encoded[port] = X_test_encoded['port_list'].map(lambda port_list: int(port in port_list))

In [ ]:
X_train_encoded = X_train_encoded.drop(columns='port_list')
X_valid_encoded = X_valid_encoded.drop(columns='port_list')
X_test_encoded = X_test_encoded.drop(columns='port_list')

In [ ]:
X_train_encoded.head()

,attack_count_per_ip,sin_day_of_week,sin_hour,same_country,port_count,watcher_country_0,watcher_country_1,watcher_country_2,watcher_country_3,watcher_country_4,...,8080/tcp,161/udp,3306/tcp,2000/tcp,22/tcp,53/udp,110/tcp,587/tcp,21/tcp,465/tcp
attacker_ip_enum,,,,,,,,,,,,,,,,,,,,,
164114,34,-0.9749,-0.2246,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
86950,159,-0.7095,-0.1465,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
132929,60,0.4204,0.2399,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165613,30,0.3494,0.4337,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63880,22,0.7240,-0.4344,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for port in top_port:
  X_train_encoded[port] = X_train_encoded['port_list'].map(lambda port_list: int(port in port_list))
  X_valid_encoded[port] = X_valid_encoded['port_list'].map(lambda port_list: int(port in port_list))
  X_test_encoded[port] = X_test_encoded['port_list'].map(lambda port_list: int(port in port_list))

In [ ]:
X_train_encoded = X_train_encoded.drop(columns='port_list')
X_valid_encoded = X_valid_encoded.drop(columns='port_list')
X_test_encoded = X_test_encoded.drop(columns='port_list')

# Random Forest

## Random Forest con parámetros default

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**Parámetros Default**

In [ ]:
rf_default = RandomForestClassifier(random_state=120)

In [ ]:
rf_default.fit(X_train_encoded, y_train)

RandomForestClassifier(random_state=120)

In [ ]:
from sklearn.metrics import f1_score

y_pred_default = rf_default.predict(X_valid_encoded)

f1_score(y_valid, y_pred_default)

0.5749718151071025

In [ ]:
y_pred_train_default = rf_default.predict(X_train_encoded)

f1_score(y_train, y_pred_train_default)

0.9997812294902646

##Ingresando Parámetros Manualmente

rf_manual = RandomForestClassifier(max_depth=110,  max_features=None, min_samples_leaf=2,
		                       min_samples_split=5, n_estimators=1400, random_state=15)


In [ ]:
rf_manual = RandomForestClassifier(max_features=None,
                                   min_samples_leaf=2,
                                   min_samples_split=5,
                                   n_estimators=1400,
                                   random_state=15)

In [ ]:
rf_manual.fit(X_train_encoded, y_train)

RandomForestClassifier(max_features=None, min_samples_leaf=2,
                       min_samples_split=5, n_estimators=1400, random_state=15)

In [ ]:
y_pred_manual = rf_manual.predict(X_valid_encoded)
f1_score(y_valid, y_pred_manual)

0.6386036960985626

In [ ]:
y_pred_train_manual = rf_manual.predict(X_train_encoded)
f1_score(y_train, y_pred_train_manual)

0.9177408014998827

##Buscando Paránetros Mediante Random Search

In [ ]:
rf = RandomForestClassifier()

In [ ]:
parameters = {
    'n_estimators': [1500, 3000],
    'max_features': [None],
    'max_depth': [10, 12, 14],
    'bootstrap': [True]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(estimator = rf,
                                   param_distributions=parameters,
                                   scoring='f1',
                                   n_iter=5,
                                   cv=2,
                                   random_state=12)

In [ ]:
random_search.fit(X_train_encoded, y_train)

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [10, 12, 14],
                                        'max_features': [None],
                                        'n_estimators': [1500, 3000]},
                   random_state=12, scoring='f1')

In [ ]:
best_model = random_search.best_estimator_

In [ ]:
best_model.fit(X_train_encoded, y_train)

RandomForestClassifier(max_depth=14, max_features=None, n_estimators=3000)

In [ ]:
y_pred = best_model.predict(X_valid_encoded)

f1_score(y_valid, y_pred)

0.6164948453608248

In [ ]:
y_pred_train = best_model.predict(X_train_encoded)

f1_score(y_train, y_pred_train)

0.8827490132342698